In [34]:
import pandas as pd
from source import data_import
from source.calculate_mean_psd import calculate_mean_psd
from source.constants import CHANNELS, FREQUENCY_RANGES

print(f"loaded channel names: {CHANNELS}")

loaded channel names: ['F4-C4', 'F3-C3', 'FT9-FT10', 'FZ-CZ', 'F7-T7', 'FP2-F4', 'T8-P8-1', 'T8-P8-0', 'FP1-F3', 'CZ-PZ']


In [35]:
# load data
df_pp = data_import.load_pyarrow(file_name="preprocessed_df")
df_pp.shape

/home/weasel/reps/ai-seizure-detectives/source/../data/preprocessed_df.arrow was loaded.


(5826809, 15)

## Feature Extraction

In [36]:
# ignore for aggregation
ignore_col = ['is_seizure']
ignore_col = []

In [37]:
### aggregate Functions for mean psd:
delta = lambda x: calculate_mean_psd(x, frequency_ranges={'Delta' : FREQUENCY_RANGES['Delta']})[x.name]['Delta']
theta = lambda x: calculate_mean_psd(x, frequency_ranges={'Theta' : FREQUENCY_RANGES['Theta']})[x.name]['Theta']
gamma = lambda x: calculate_mean_psd(x, frequency_ranges={'Gamma': FREQUENCY_RANGES['Gamma']})[x.name]['Gamma']

delta_agg = pd.NamedAgg(column='delta', aggfunc=delta)
theta_agg = pd.NamedAgg(column='theta', aggfunc=theta)
gamma_agg = pd.NamedAgg(column='gamma', aggfunc=gamma)

In [38]:
df_features = df_pp.groupby(['seizure_id', 'segment_id', "is_seizure"]).agg(
    {C:[
        'mean', 
        'std',
        'var',
        delta_agg,
        theta_agg,
        gamma_agg
        ] for C in CHANNELS} | 
    {ic: ['first'] for ic in ignore_col}) # just taking first element for target column
df_features.head()

channel                              F4-C4                                     \
                                      mean        std         var       delta   
seizure_id segment_id is_seizure                                                
0          1          False       0.642823  25.278278  638.991344   99.724544   
           2          False       0.459926  24.908361  620.426452   80.467433   
           3          False      -0.336786  25.078313  628.921793   97.815139   
           4          False       0.571550  23.796227  566.260437   78.057846   
           5          False      -0.156618  28.030262  785.695596  102.786435   

channel                                                   F3-C3             \
                                      theta     gamma      mean        std   
seizure_id segment_id is_seizure                                             
0          1          False       33.651173  0.131576 -0.136380  28.577184   
           2          False       30.739370  0.113581  0.607864  22.789379   
           3          False       19.120020  0.108965  0.020483  24.208610   
           4          False       22.859691  0.112154  0.170771  27.059930   
           5          False       31.697164  0.123064  0.019398  28.681243   

channel                                                   ...      FP1-F3  \
                                         var       delta  ...         var   
seizure_id segment_id is_seizure                          ...               
0          1          False       816.655466  123.466899  ...  704.741392   
           2          False       519.355782   69.215516  ...  654.580741   
           3          False       586.056809   92.561775  ...  505.928076   
           4          False       732.239825  112.071663  ...  623.521637   
           5          False       822.613674  124.450518  ...  838.658633   

channel                                                               CZ-PZ  \
                                       delta      theta     gamma      mean   
seizure_id segment_id is_seizure                                              
0          1          False       115.680103  29.233559  0.343249  0.758870   
           2          False        98.999296  24.519002  0.363930  0.040398   
           3          False        76.273119  21.891098  0.347473  0.139786   
           4          False       111.418632  24.584840  0.326450  0.347514   
           5          False       115.905093  37.751880  0.323275 -0.032786   

channel                                                               \
                                        std          var       delta   
seizure_id segment_id is_seizure                                       
0          1          False       35.447713  1256.540374  194.364620   
           2          False       33.861154  1146.577756  187.417290   
           3          False       30.472674   928.583879  117.606520   
           4          False       30.803236   948.839359  158.377845   
           5          False       39.120852  1530.441051  215.122132   

channel                                                
                                      theta     gamma  
seizure_id segment_id is_seizure                       
0          1          False       76.886992  0.174189  
           2          False       34.401955  0.181605  
           3          False       31.998680  0.155509  
           4          False       31.605055  0.181355  
           5          False       60.283108  0.170928  

[5 rows x 60 columns]

## Flatten Dataframe

In [39]:
# joining column names with agg functions, but leaving target 'is_seizure' column as 'is_seizure'.
df_features.columns = ['_'.join(col).strip() for col in df_features.columns.values if 'is_seizure' != col[0]] #+ ['is_seizure']
df_features.reset_index(inplace=True)
df_features.tail(30)


,seizure_id,segment_id,is_seizure,F4-C4_mean,F4-C4_std,F4-C4_var,F4-C4_delta,F4-C4_theta,F4-C4_gamma,F3-C3_mean,...,FP1-F3_var,FP1-F3_delta,FP1-F3_theta,FP1-F3_gamma,CZ-PZ_mean,CZ-PZ_std,CZ-PZ_var,CZ-PZ_delta,CZ-PZ_theta,CZ-PZ_gamma
1108,58,7,False,-0.129867,18.902304,357.297092,43.050805,27.537749,0.187851,-1.821281,...,1201.368801,151.193533,19.045718,0.343143,-0.016320,22.489849,505.793304,38.784601,43.715104,0.142116
1109,58,8,False,0.768381,17.562436,308.439155,33.648257,21.871237,0.181689,2.967463,...,379.393423,40.342519,20.953553,0.300189,0.539300,24.665634,608.393500,43.943237,61.133626,0.145097
1110,58,9,False,0.055482,17.866366,319.207030,32.340037,22.670158,0.162112,-0.140394,...,502.003677,71.530600,31.945473,0.309519,0.039282,26.121613,682.338657,62.158532,71.514952,0.132452
1111,58,10,False,-0.110975,29.447813,867.173674,40.970434,28.339451,6.148782,-0.022131,...,695.800465,72.005667,26.834090,0.898667,0.191656,23.714588,562.381698,49.758678,50.889884,0.233121
1112,58,11,False,0.411730,15.692893,246.266880,28.056838,18.262682,0.166530,0.560296,...,349.773758,46.062897,27.492809,0.213767,0.173616,18.091766,327.311980,27.480825,34.086131,0.149297
1113,58,12,False,0.698367,33.805748,1142.828615,79.006955,21.153684,4.131121,1.144526,...,2558.095794,257.892151,23.174721,1.266160,0.639870,39.510151,1561.052071,140.890543,30.209887,0.643544
1114,58,13,False,0.105825,49.455717,2445.867908,109.618573,36.115524,24.873477,-0.466066,...,2460.796850,204.952106,33.671004,8.519916,0.124016,25.087019,629.358499,64.126837,44.241002,0.587274
1115,58,14,False,-1.073148,24.568884,603.630037,39.848499,26.372584,3.444726,-1.008743,...,1035.948034,116.835295,20.371412,0.912723,-0.461689,19.933175,397.331484,39.078317,35.143902,0.186331
1116,58,15,False,0.821020,26.150749,683.861694,78.725586,35.386311,1.601313,1.176248,...,2316.850655,318.915036,114.556493,1.303635,0.448621,24.846112,617.329278,68.143516,54.886177,0.265559
1117,59,1,False,0.144024,22.991423,528.605555,41.134349,16.197523,3.893597,-0.024945,...,698.845037,52.493464,15.564249,7.029091,0.176004,20.563030,422.838189,54.545782,23.015469,0.319742


In [40]:
from source.data_import import save_pyarrow

save_pyarrow(df_features, file_name="feature_extracted")

/home/weasel/reps/ai-seizure-detectives/source/../data/feature_extracted.arrow was successfully written.
